# Coursera Capstone

## Final report

Opening a New Shopping Mall in Kuala Lumpur, Malaysia

- Build a dataframe of neighborhoods in Saint Petersburg, Russua by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new sushi bar

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import re
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

# Configuration

In [3]:
city = 'Saint Petersburg'
country = 'Russia'
districts_url = "https://en.wikipedia.org/wiki/Category:Districts_of_Saint_Petersburg"

In [4]:
# send the GET request
data = requests.get(districts_url).text

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-content-ltr")[0].findAll("li"):
    neighborhoodList.append(row.text)
# create a new DataFrame from the list
spb_df = pd.DataFrame({"Neighborhood": neighborhoodList})
# remove redundand information
spb_df['Neighborhood'].replace(r'\(.*\)', '', regex=True, inplace=True)
spb_df['Neighborhood'] = spb_df['Neighborhood'].apply(lambda x: x.strip('►').split(',')[0])
print(spb_df.shape)
spb_df

(18, 1)


,Neighborhood
0,Admiralteysky District
1,Frunzensky District
2,Kalininsky District
3,Kirovsky District
4,Krasnogvardeysky District
5,Krasnoselsky District
6,Kurortny District‎
7,Moskovsky District
8,Nevsky District‎
9,Petrodvortsovy District‎


In [7]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, {}, {}'.format(neighborhood, city, country))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [8]:
coords = [ get_latlng(neighborhood) for neighborhood in spb_df["Neighborhood"].tolist() ]
coords

[[59.92659000000003, 30.305600000000027],
 [59.90066000000007, 30.35211000000004],
 [59.99628000000007, 30.380810000000054],
 [59.87876000000006, 30.267210000000034],
 [59.96040000000005, 30.444180000000074],
 [59.843210000000056, 30.172190000000057],
 [60.14784000000003, 30.010700000000043],
 [59.85352000000006, 30.329800000000034],
 [59.91309000000007, 30.47637000000003],
 [59.88201000000004, 29.89546000000007],
 [59.96273000000008, 30.314520000000073],
 [59.99968000000007, 30.23428000000007],
 [59.71229000000005, 30.31000000000006],
 [59.93268000000006, 30.348100000000045],
 [59.93703000000005, 30.27570000000003],
 [59.77076000000005, 30.594020000000057],
 [60.012110000000064, 29.723330000000033],
 [60.01015000000007, 30.348060000000032]]

In [9]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
spb_df['Latitude'] = df_coords['Latitude']
spb_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
spb_df

,Neighborhood,Latitude,Longitude
0,Admiralteysky District,59.92659,30.30560
1,Frunzensky District,59.90066,30.35211
2,Kalininsky District,59.99628,30.38081
3,Kirovsky District,59.87876,30.26721
4,Krasnogvardeysky District,59.96040,30.44418
5,Krasnoselsky District,59.84321,30.17219
6,Kurortny District‎,60.14784,30.01070
7,Moskovsky District,59.85352,30.32980
8,Nevsky District‎,59.91309,30.47637
9,Petrodvortsovy District‎,59.88201,29.89546


In [10]:
# save the DataFrame as CSV file
spb_df.to_csv(f"{city}_df.csv", index=False)

In [11]:
# get the coordinates of Saint Petersburg
address = f'{city}, {country}'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of {city}, {country} {latitude}, {longitude}.')

The geograpical coordinate of Saint Petersburg, Russia 59.938732, 30.316229.


In [12]:
# create map of Saint Petersburg using latitude and longitude values
map_spb = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(spb_df['Latitude'], spb_df['Longitude'], spb_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_spb)  
    
map_spb

In [13]:
# save the map as HTML file
map_spb.save(f"map_{city.replace(' ', '_')}.html")

In [14]:
# define Foursquare Credentials and Version
with open('creds') as f:
    CLIENT_ID = f.readline().strip()  # your Foursquare ID
    CLIENT_SECRET = f.readline().strip()  # your Foursquare Secret
    VERSION = f.readline().strip()  # Foursquare API version

In [16]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(spb_df['Latitude'], spb_df['Longitude'], spb_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [17]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1439, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Admiralteysky District,59.92659,30.3056,Булочная Ф. Вольчека,59.926702,30.307921,Bakery
1,Admiralteysky District,59.92659,30.3056,"Chao, mama!",59.926993,30.308474,Hotel
2,Admiralteysky District,59.92659,30.3056,CUP IN CUP,59.928074,30.302705,Coffee Shop
3,Admiralteysky District,59.92659,30.3056,ЛУУК,59.926154,30.310403,Clothing Store
4,Admiralteysky District,59.92659,30.3056,Pacman,59.923537,30.307985,Hookah Bar


In [37]:
# save the DataFrame as CSV file
venues_df.to_csv(f"{city}_venues.csv", index=False)

In [18]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Admiralteysky District,100,100,100,100,100,100
Frunzensky District,96,96,96,96,96,96
Kalininsky District,100,100,100,100,100,100
Kirovsky District,96,96,96,96,96,96
Krasnogvardeysky District,100,100,100,100,100,100
Krasnoselsky District,100,100,100,100,100,100
Kurortny District‎,18,18,18,18,18,18
Moskovsky District,100,100,100,100,100,100
Nevsky District‎,100,100,100,100,100,100


In [19]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 251 uniques categories.


In [21]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:20]

array(['Bakery', 'Hotel', 'Coffee Shop', 'Clothing Store', 'Hookah Bar',
       'Café', 'Bar', 'Opera House', 'Garden', 'Palace', 'Pizza Place',
       'Arcade', 'Park', 'Italian Restaurant', 'Plaza', 'Hostel',
       'Restaurant', 'Concert Hall', 'Music Venue', 'Historic Site'],
      dtype=object)

In [22]:
# check if the results contain "Bakery"
"Sushi Restaurant" in venues_df['VenueCategory'].unique()

True

In [23]:
# one hot encoding
spb_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
spb_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [spb_onehot.columns[-1]] + list(spb_onehot.columns[:-1])
spb_onehot = spb_onehot[fixed_columns]

print(spb_onehot.shape)
spb_onehot.head()

(1439, 252)


,Neighborhoods,Accessories Store,Adult Boutique,Advertising Agency,Airport,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Blini House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Boxing Gym,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Stop,Cafeteria,Café,Camera Store,Canal,Car Wash,Caucasian Restaurant,Cemetery,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Country Dance Club,Coworking Space,Cupcake Shop,Cycle Studio,Dairy Store,Dance Studio,Department Store,Design Studio,Dessert Shop,Diner,Dive Spot,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fishing Spot,Flower Shop,Food,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nightclub,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paintball Field,Palace,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Perfume Shop,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Roller Rink,Russian Restaurant,Salon / Barbershop,Salsa Club,Sandwich Place,Sauna / Steam Room,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Volleyball Court,Warehouse Store,Watch Shop,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Admiralteysky District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Admiralteysky District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [24]:
spb_grouped = spb_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(spb_grouped.shape)
spb_grouped

(18, 252)


,Neighborhoods,Accessories Store,Adult Boutique,Advertising Agency,Airport,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Blini House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Boxing Gym,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Stop,Cafeteria,Café,Camera Store,Canal,Car Wash,Caucasian Restaurant,Cemetery,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Country Dance Club,Coworking Space,Cupcake Shop,Cycle Studio,Dairy Store,Dance Studio,Department Store,Design Studio,Dessert Shop,Diner,Dive Spot,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fishing Spot,Flower Shop,Food,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nightclub,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paintball Field,Palace,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Perfume Shop,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Roller Rink,Russian Restaurant,Salon / Barbershop,Salsa Club,Sandwich Place,Sauna / Steam Room,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tea Room,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Volleyball Court,Warehouse Store,Watch Shop,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Admiralteysky District,0.00,0.00,0.00,0.000000,0.00,0.04,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.050000,0.00,0.030000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.00,0.00,0.000000,0.00,0.01,0.00,0.000000,0.040000,0.00,0.01,0.00,0.020000,0.000000,0.01,0.01,0.050000,0.010000,0.040000,0.00,0.00,0.00,0.00,0.000000,0.030000,0.00,0.000000,0.00,0.00,0.00,0.00,0.01,0.00,0.010000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.01,0.010000,0.00,0.00,0.01,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0

In [25]:
len(spb_grouped[spb_grouped["Sushi Restaurant"] > 0])

10

In [26]:
spb_mall = spb_grouped[["Neighborhoods","Sushi Restaurant"]]
spb_mall

,Neighborhoods,Sushi Restaurant
0,Admiralteysky District,0.000000
1,Frunzensky District,0.010417
2,Kalininsky District,0.010000
3,Kirovsky District,0.020833
4,Krasnogvardeysky District,0.010000
5,Krasnoselsky District,0.020000
6,Kurortny District‎,0.000000
7,Moskovsky District,0.020000
8,Nevsky District‎,0.010000
9,Petrodvortsovy District‎,0.000000


In [38]:
# set number of clusters
kclusters = 3

spb_clustering = spb_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(spb_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 2, 1, 2, 1, 0, 1, 2, 0])

In [39]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
spb_merged = spb_mall.copy()

# add clustering labels
spb_merged["Cluster Labels"] = kmeans.labels_

spb_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
spb_merged

,Neighborhood,Sushi Restaurant,Cluster Labels
0,Admiralteysky District,0.000000,0
1,Frunzensky District,0.010417,2
2,Kalininsky District,0.010000,2
3,Kirovsky District,0.020833,1
4,Krasnogvardeysky District,0.010000,2
5,Krasnoselsky District,0.020000,1
6,Kurortny District‎,0.000000,0
7,Moskovsky District,0.020000,1
8,Nevsky District‎,0.010000,2
9,Petrodvortsovy District‎,0.000000,0


In [40]:
# merge two data frames to add latitude/longitude for each neighborhood
spb_merged = spb_df.join(spb_merged.set_index("Neighborhood"), on="Neighborhood")

print(spb_merged.shape)
spb_merged # check the last columns!

(18, 5)


,Neighborhood,Latitude,Longitude,Sushi Restaurant,Cluster Labels
0,Admiralteysky District,59.92659,30.30560,0.000000,0
1,Frunzensky District,59.90066,30.35211,0.010417,2
2,Kalininsky District,59.99628,30.38081,0.010000,2
3,Kirovsky District,59.87876,30.26721,0.020833,1
4,Krasnogvardeysky District,59.96040,30.44418,0.010000,2
5,Krasnoselsky District,59.84321,30.17219,0.020000,1
6,Kurortny District‎,60.14784,30.01070,0.000000,0
7,Moskovsky District,59.85352,30.32980,0.020000,1
8,Nevsky District‎,59.91309,30.47637,0.010000,2
9,Petrodvortsovy District‎,59.88201,29.89546,0.000000,0


In [41]:
# sort the results by Cluster Labels
print(spb_merged.shape)
spb_merged.sort_values(["Cluster Labels"], inplace=True)
spb_merged

(18, 5)


,Neighborhood,Latitude,Longitude,Sushi Restaurant,Cluster Labels
0,Admiralteysky District,59.92659,30.30560,0.000000,0
15,Kolpinsky District,59.77076,30.59402,0.000000,0
14,Vasileostrovsky District‎,59.93703,30.27570,0.000000,0
6,Kurortny District‎,60.14784,30.01070,0.000000,0
16,Kronshtadtsky District,60.01211,29.72333,0.000000,0
9,Petrodvortsovy District‎,59.88201,29.89546,0.000000,0
10,Petrogradsky District‎,59.96273,30.31452,0.000000,0
12,Pushkinsky District,59.71229,30.31000,0.000000,0
3,Kirovsky District,59.87876,30.26721,0.020833,1
5,Krasnoselsky District,59.84321,30.17219,0.020000,1


In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(spb_merged['Latitude'], spb_merged['Longitude'], spb_merged['Neighborhood'], spb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [44]:
spb_merged.loc[spb_merged['Cluster Labels'] == 0]

,Neighborhood,Latitude,Longitude,Sushi Restaurant,Cluster Labels
0,Admiralteysky District,59.92659,30.30560,0.0,0
15,Kolpinsky District,59.77076,30.59402,0.0,0
14,Vasileostrovsky District‎,59.93703,30.27570,0.0,0
6,Kurortny District‎,60.14784,30.01070,0.0,0
16,Kronshtadtsky District,60.01211,29.72333,0.0,0
9,Petrodvortsovy District‎,59.88201,29.89546,0.0,0
10,Petrogradsky District‎,59.96273,30.31452,0.0,0
12,Pushkinsky District,59.71229,30.31000,0.0,0


In [45]:
spb_merged.loc[spb_merged['Cluster Labels'] == 1]

,Neighborhood,Latitude,Longitude,Sushi Restaurant,Cluster Labels
3,Kirovsky District,59.87876,30.26721,0.020833,1
5,Krasnoselsky District,59.84321,30.17219,0.020000,1
7,Moskovsky District,59.85352,30.32980,0.020000,1
11,Primorsky District,59.99968,30.23428,0.020000,1


In [46]:
spb_merged.loc[spb_merged['Cluster Labels'] == 2]

,Neighborhood,Latitude,Longitude,Sushi Restaurant,Cluster Labels
13,Tsentralny District,59.93268,30.34810,0.010000,2
8,Nevsky District‎,59.91309,30.47637,0.010000,2
2,Kalininsky District,59.99628,30.38081,0.010000,2
1,Frunzensky District,59.90066,30.35211,0.010417,2
4,Krasnogvardeysky District,59.96040,30.44418,0.010000,2
17,Vyborgsky District,60.01015,30.34806,0.010000,2


#### Observations